In [1]:
import warnings
warnings.filterwarnings("ignore")

#######################################################################################
import matplotlib
import matplotlib.pyplot as plt
#matplotlib內建不支援中文，解決辦法：每次預先指定字體
matplotlib.rcParams.update({'font.size': 36})
matplotlib.use('qt4agg')
myfont = matplotlib.font_manager.FontProperties(fname='C:\\Windows\\Fonts\\msjh.ttc')
#指定字體
matplotlib.rcParams['axes.unicode_minus']=False

from matplotlib.ticker import FormatStrFormatter
majorFormatter = FormatStrFormatter('%0.f') #設定圖表浮點數的格式
%matplotlib inline 
#繪圖完就直接顯示該圖，省略每次繪圖完都要輸入plt.show指令的動作
#######################################################################################
import seaborn as sns

import operator
import numpy as np
# 設定array的float格式
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
import pandas as pd
pd.set_option('display.max_columns', 100)  #設定可顯示欄位的上限
pd.options.display.float_format = '{:,.6f}'.format  #設定浮點數的格式

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


# 資料讀取

In [3]:
data = pd.read_excel('data.xlsx', encoding='utf8')

In [4]:
data = data.fillna(0)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44236 entries, 0 to 44235
Columns: 102 entries, account to num_20180609
dtypes: int64(101), object(1)
memory usage: 34.4+ MB


In [5]:
data.head(10)

,account,num_20180301,num_20180302,num_20180303,num_20180304,num_20180305,num_20180306,num_20180307,num_20180308,num_20180309,num_20180310,num_20180311,num_20180312,num_20180313,num_20180314,num_20180315,num_20180316,num_20180317,num_20180318,num_20180319,num_20180320,num_20180321,num_20180322,num_20180323,num_20180324,num_20180325,num_20180326,num_20180327,num_20180328,num_20180329,num_20180330,num_20180331,num_20180401,num_20180402,num_20180403,num_20180404,num_20180405,num_20180406,num_20180407,num_20180408,num_20180409,num_20180410,num_20180411,num_20180412,num_20180413,num_20180414,num_20180415,num_20180416,num_20180417,num_20180418,...,num_20180421,num_20180422,num_20180423,num_20180424,num_20180425,num_20180426,num_20180427,num_20180428,num_20180429,num_20180430,num_20180501,num_20180502,num_20180503,num_20180504,num_20180505,num_20180506,num_20180507,num_20180508,num_20180509,num_20180510,num_20180511,num_20180512,num_20180513,num_20180514,num_20180515,num_20180516,num_20180517,num_20180518,num_20180519,num_20180520,num_20180521,num_20180522,num_20180523,num_20180524,num_20180525,num_20180526,num_20180527,num_20180528,num_20180529,num_20180530,num_20180531,num_20180601,num_20180602,num_20180603,num_20180604,num_20180605,num_20180606,num_20180607,num_20180608,num_20180609
0,kiss9985,1,0,0,1,0,1,1,1,1,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,1,1,1,0,0,1,...,0,1,0,0,1,1,0,0,1,0,1,1,1,0,0,0,1,0,1,1,1,0,0,1,0,1,1,1,0,1,1,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0
1,kitming20,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,kitty1100,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,kitty77688,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,kjfgw6305,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,kjlghjbdb,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,kk119gm,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,kkaqr033,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,0,0,1,1,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,1
8,kkboy3001,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1
9,klkjlgc,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


# 建立訓練樣本

In [6]:
#訓練樣本(X)，過去30天是否登入
train_df = data.iloc[:,-37:-7].copy()
train_df.head()

,num_20180504,num_20180505,num_20180506,num_20180507,num_20180508,num_20180509,num_20180510,num_20180511,num_20180512,num_20180513,num_20180514,num_20180515,num_20180516,num_20180517,num_20180518,num_20180519,num_20180520,num_20180521,num_20180522,num_20180523,num_20180524,num_20180525,num_20180526,num_20180527,num_20180528,num_20180529,num_20180530,num_20180531,num_20180601,num_20180602
0,0,0,0,1,0,1,1,1,0,0,1,0,1,1,1,0,1,1,0,1,0,1,1,0,0,0,1,1,0,1
1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
#預測對象(Y)，未來一週是否登入
train_y = data.iloc[:, -7:].copy()
train_y.head()

,num_20180603,num_20180604,num_20180605,num_20180606,num_20180607,num_20180608,num_20180609
0,1,1,1,0,0,0,0
1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1
4,0,0,0,0,0,0,0


In [8]:
#GRU與lstm同樣要求輸入必須為3維，於是利用np.reshape函數將其轉換為3維
train_value = train_df.values
train_value = np.reshape(train_value, (train_value.shape[0], train_value.shape[1], 1))
print(train_value.shape) #樣本數, timesteps, 特徵數

(44236, 30, 1)


## 調整模型參數

In [9]:
#node代表各層神經元的數量，dropout_rate代表drop的比率，opt代表優化器

def get_model(node=128, dropout_rate=0, opt='nadam'):
    model = Sequential()
    model.add(GRU(units=node,input_shape=(train_value.shape[1], train_value.shape[2]), return_sequences=True))#timestep, feature
    model.add(Dropout(dropout_rate)) #依一定比例斷開神經元連結，降低模型複雜度，避免過擬合
    model.add(GRU(units=node))
    model.add(Dropout(dropout_rate))
    model.add(Dense(7, activation='sigmoid')) #模型輸出未來7天的登入機率
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model #回傳keras模型

In [10]:
#KerasClassifier的作用是將模型包裝成sklearn的分類器，如此一來就能使用sklearn中自動調整參數的GridSearchCV

#build_fn參數放入函數，該函數必須回傳一個keras模型
neural_network = KerasClassifier(build_fn=get_model, batch_size=1024,epochs=10,verbose=0)

In [11]:
#調整神經元數量、dropout比率及優化器

nodes = [64, 256]
dropout_rates = [0.1,0.2,0.3,0.4]
opts = ['nadam', 'adam', 'rmsprop', 'sgd']

hyperparameters = dict(node=nodes, dropout_rate=dropout_rates, opt=opts) #必須為字典格式

In [12]:
#建立GridSearchCV
#estimator放入分類器
#param_grid放入事先設定好的參數範圍

grid = GridSearchCV(estimator=neural_network, param_grid=hyperparameters)

In [13]:
#GridSearchCV開始訓練
#grid_result = grid.fit(train_value, train_y, verbose=1) #verbose設定為1可以看到詳細訓練過程

In [15]:
#顯示最佳參數及其在交叉驗證的表現
#print('Best: %f using %s' %(grid_result.best_score_, grid_result.best_params_))

In [17]:
#套用GridSearchCV找出的參數
#model = get_model(**grid_result.best_params_)
model = get_model(dropout_rate=0.3, node=256, opt='nadam')
model.fit(train_value, train_y , epochs=60,batch_size=512, verbose=1)

Epoch 1/60
44236/44236 [==============================] - 81s 2ms/step - loss: 0.2630 - acc: 0.9073
Epoch 2/60
44236/44236 [==============================] - 81s 2ms/step - loss: 0.2222 - acc: 0.9167
Epoch 3/60
44236/44236 [==============================] - 81s 2ms/step - loss: 0.2101 - acc: 0.9203
Epoch 4/60
44236/44236 [==============================] - 79s 2ms/step - loss: 0.2049 - acc: 0.9228
Epoch 5/60
44236/44236 [==============================] - 76s 2ms/step - loss: 0.2013 - acc: 0.9246
Epoch 6/60
44236/44236 [==============================] - 76s 2ms/step - loss: 0.1971 - acc: 0.9268
Epoch 7/60
44236/44236 [==============================] - 77s 2ms/step - loss: 0.1939 - acc: 0.9282
Epoch 8/60
44236/44236 [==============================] - 82s 2ms/step - loss: 0.1908 - acc: 0.9298
Epoch 9/60
44236/44236 [==============================] - 78s 2ms/step - loss: 0.1872 - acc: 0.9315
Epoch 10/60
44236/44236 [==============================] - 76s 2ms/step - loss: 0.1844 - acc: 0.9327

In [18]:
train_y_pred_lstm = model.predict(train_value)

In [19]:

pred_df = pd.DataFrame(train_y_pred_lstm) #train_y_pred_lstm為np.array將其轉換成pandas的dataframe

for i in range(7):
    pred_df[i] = np.round(pred_df[i])

pred_df['sum'] = pred_df[0]+pred_df[1]+pred_df[2]+pred_df[3]+pred_df[4]+pred_df[5]+pred_df[6]
pred_df.head()


,0,1,2,3,4,5,6,sum
0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000
2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000
3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:

result = train_y.copy()
result['遊戲帳號'] = data.account
result['登入次數實際值'] = result.iloc[:,0]+result.iloc[:,1]+result.iloc[:,2]+result.iloc[:,3]+result.iloc[:,4]+result.iloc[:,5]+result.iloc[:,6]

result['下週登入次數預測']  = pred_df['sum']
result = result[['遊戲帳號','登入次數實際值', '下週登入次數預測']]
result.head(20)


,遊戲帳號,登入次數實際值,下週登入次數預測
0,kiss9985,3,3.000000
1,kitming20,7,7.000000
2,kitty1100,7,7.000000
3,kitty77688,7,7.000000
4,kjfgw6305,0,0.000000
5,kjlghjbdb,7,7.000000
6,kk119gm,0,0.000000
7,kkaqr033,5,5.000000
8,kkboy3001,6,6.000000
9,klkjlgc,7,7.000000


In [21]:

y_true = result.登入次數實際值
y_pred = result.下週登入次數預測

#用RMSE衡量模型在訓練樣本上的表現
score = np.sqrt(mean_squared_error(y_true, y_pred))
score


0.7207383610939302

In [22]:
#預測準確率
score = accuracy_score(y_true, y_pred)
score

0.8794194773487657